In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Beispiel für die Feiertage in den nächsten 5 Jahren (2020-2024)
years = [2020, 2021, 2022, 2023, 2024]
holiday_data = []
for year in years:
    holiday_data.extend(generate_holidays(year))

# Lade die vorhandenen Feiertagsdaten und erstelle das Modell wie vorher
df = pd.read_csv('kombinierte_feiertage_wetter.csv', delimiter=',')
df.columns = ['Datum', 'Wochentag', 'Feiertag', 'Temperatur', 'Werte_2', 'Werte_3', 'Werte_4']
df['Datum'] = pd.to_datetime(df['Datum'], format='%Y-%m-%d')

# Fehlende Werte behandeln
df = df.dropna(subset=['Temperatur'])
df['Wochentag_num'] = df['Datum'].dt.weekday  # 0=Montag, 1=Dienstag, ..., 6=Sonntag
df['Monat'] = df['Datum'].dt.month

# Eingabewerte und Ziel definieren
X = df[['Wochentag_num', 'Monat']]  # Features: Wochentag und Monat
y = df['Temperatur']  # Ziel: Temperatur

# Train-Test-Split (80% Training, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineare Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersage für jeden Feiertag in den Jahren 2020 bis 2024
forecast_data = []
for holiday in holiday_data:
    # Feiertagsdatum und Name trennen
    date_str, holiday_name = holiday.split(' ', 1)
    date = pd.to_datetime(date_str)
    
    # Wochentag und Monat extrahieren
    weekday_num = date.weekday()  # Wochentag (0=Montag, ..., 6=Sonntag)
    month = date.month
    
    # Vorhersage der Temperatur mit dem Modell
    # Übergebe die Eingabewerte als DataFrame mit Spaltennamen
    prediction = model.predict(pd.DataFrame([[weekday_num, month]], columns=['Wochentag_num', 'Monat']))[0]
    
    # Ergebnis speichern
    forecast_data.append([date_str, holiday_name, prediction])

# Erstelle ein DataFrame aus den Vorhersagen
forecast_df = pd.DataFrame(forecast_data, columns=['Datum', 'Feiertag', 'Vorhergesagte_Temperatur'])

# Speichere die Ergebnisse in einer CSV-Datei
forecast_df.to_csv('feiertags_temperature_predictions_2020_2024.csv', index=False)

# Ausgabe des DataFrames für eine Ansicht
print(forecast_df.head())

        Datum             Feiertag  Vorhergesagte_Temperatur
0  2020-01-01              Neujahr                  4.857194
1  2020-04-06           Karfreitag                  4.792103
2  2020-04-09          Ostermontag                  4.890760
3  2020-05-01       Tag der Arbeit                  4.923873
4  2020-05-17  Christi Himmelfahrt                  4.989645
